In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

/Users/admin/.pyenv/versions/anaconda3-2.4.0/envs/tensorFlow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(tf.truncated_normal([5,5,num_filters1,num_filters2],stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [5]:
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*num_filters2])

num_units1 = 7 * 7 * num_filters2
num_units2 =  1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

In [6]:
t = tf.placeholder(tf.float32, [None,10])
loss = -tf.reduce_sum(t*tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p,1), tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [8]:
i = 0
# for _ in range(20000):
for _ in range(14000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [],[]
        for c in range(4):
            start = int(len(mnist.test.labels) / 4 * c)
            end = int(len(mnist.test.labels) /4 * (c + 1))
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:mnist.test.images[start:end], t:mnist.test.labels[start:end], keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_val)
        acc_val = np.mean(acc_val)      
        print('Step:%d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val) )
        saver.save(sess, './sessions/cnn-session', global_step=i)

Step:500, Loss: 244.278778, Accuracy: 0.972000
Step:1000, Loss: 134.620178, Accuracy: 0.986400
Step:1500, Loss: 112.964005, Accuracy: 0.986400
Step:2000, Loss: 98.829132, Accuracy: 0.988800
Step:2500, Loss: 81.765381, Accuracy: 0.988400
Step:3000, Loss: 68.454697, Accuracy: 0.992000
Step:3500, Loss: 67.217575, Accuracy: 0.990400
Step:4000, Loss: 62.547356, Accuracy: 0.992400
Step:4500, Loss: 58.027382, Accuracy: 0.992800
Step:5000, Loss: 44.214378, Accuracy: 0.994800
Step:5500, Loss: 55.525826, Accuracy: 0.990800
Step:6000, Loss: 56.310337, Accuracy: 0.994000
Step:6500, Loss: 47.793503, Accuracy: 0.993200
Step:7000, Loss: 47.325188, Accuracy: 0.994400
Step:7500, Loss: 35.826271, Accuracy: 0.994800
Step:8000, Loss: 46.296482, Accuracy: 0.994800
Step:8500, Loss: 43.738319, Accuracy: 0.994800
Step:9000, Loss: 41.019577, Accuracy: 0.994800
Step:9500, Loss: 52.390945, Accuracy: 0.992800
Step:10000, Loss: 40.027397, Accuracy: 0.994800
Step:10500, Loss: 39.714485, Accuracy: 0.994000
Step:1100